In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from google.colab import files
from sklearn.impute import SimpleImputer

## CARGA DE ARCHIVOS Y EXPLORACIÓN DE DATOS

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [3]:
!kaggle competitions download -c udea-ai4eng-20241

 42% 9.00M/21.2M [00:00<00:00, 88.9MB/s]
100% 21.2M/21.2M [00:00<00:00, 144MB/s] 


In [4]:
!unzip udea-ai4eng-20241.zip

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [24]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

Extraer los ID del archivo test para su posterior uso:

In [25]:
id_df = test_df['ID']

Muestra el dataset sin cambios:

In [26]:
display(train_df)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) incompleta,bajo
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,medio-bajo
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,No,N,Si,Si,No,Primaria completa,bajo


## PREPROCESADO DE DATOS

Elimina las columnas que no nos sirven para el entrenamiento:

In [27]:
columns_drop = ['ID', 'FAMI_TIENEINTERNET.1']
train_df = train_df.drop(columns_drop, axis = 1)
test_df = test_df.drop(columns_drop, axis = 1)

Elimina las filas que contienen al menos un valor nulo:

In [28]:
# Evalúa si hay valores nulos en el dataset y los imputa
def impute_missing_values(df):
    imputer = SimpleImputer(strategy='most_frequent')
    df_imputed = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)
    return df_imputed

train_df = impute_missing_values(train_df)
test_df = impute_missing_values(test_df)

In [29]:
display(train_df)
display(test_df)

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Postgrado,medio-alto
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,Técnica o tecnológica incompleta,bajo
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Secundaria (Bachillerato) completa,bajo
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Secundaria (Bachillerato) completa,alto
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Primaria completa,medio-bajo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto
692496,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,No,N,No,Si,Secundaria (Bachillerato) incompleta,bajo
692497,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Secundaria (Bachillerato) incompleta,medio-bajo
692498,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,No,N,Si,Si,Primaria completa,bajo


,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE
0,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Primaria completa
1,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Técnica o tecnológica completa
2,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Secundaria (Bachillerato) completa
3,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,Primaria incompleta
4,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Secundaria (Bachillerato) completa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 1,Si,Primaria incompleta,Si,Si,N,Si,Si,Primaria incompleta
296782,20183,DERECHO,META,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Si,No,N,No,Si,Educación profesional completa
296783,20212,DERECHO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Educación profesional completa
296784,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,Entre 5.5 millones y menos de 7 millones,Entre 11 y 20 horas,Estrato 3,Si,Educación profesional completa,Si,No,N,No,Si,Educación profesional completa


Establece las características y etiqueta correspondientes

In [30]:
# Separa las características (X) y etiqueta (y) en los datos de entrenamiento
X_train = train_df.drop('RENDIMIENTO_GLOBAL', axis=1)
y_train = train_df['RENDIMIENTO_GLOBAL']
# Establece las características en los datos de prueba
X_test = test_df

Codifica y_train para manejarla posteriormente sin inconvenientes de tipo de dato:

In [31]:
# Codifica la etiqueta categórica en y_train
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

Enlistar las columnas categóricas:

In [32]:
# Selección de columnas categóricas
categorical_features = X_train.columns.tolist()

Aplica conversión one-hot para cada columna:

In [33]:
# Preprocesamiento One-Hot-Encoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

## ENTRENAMIENTO RANDOM FOREST

Pipeline después del preprocesado y el modelo de entrenamiento a implementar (en este caso es Random Forest Classifier):

In [34]:
# Crear pipeline con el preprocesador y el modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=5, n_jobs=-1, random_state=42))
])

In [35]:
# Entrenar el modelo
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['PERIODO',
                                                   'ESTU_PRGM_ACADEMICO',
                                                   'ESTU_PRGM_DEPARTAMENTO',
                                                   'ESTU_VALORMATRICULAUNIVERSIDAD',
                                                   'ESTU_HORASSEMANATRABAJA',
                                                   'FAMI_ESTRATOVIVIENDA',
                                                   'FAMI_TIENEINTERNET',
                                                   'FAMI_EDUCACIONPADRE',
                                                   'FAMI_TIENELAVADORA',
                                                   'FAMI_TIENEAUTOMOVIL',
                                                   'ESTU_PRIVADO_LIBERTAD',
                                                   'ESTU_PAGOMATRICULAPROPIO',
                                                   'FAMI_TIENECOMPUTADOR',
                                                   'FAMI_EDUCACIONMADRE'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=5, n_jobs=-1,
                                        random_state=42))])

## TESTEO DEL MODELO PREDICTIVO

Probar el modelo predictivo previamente entrenado:

In [42]:
# Hacer predicciones
predictions = model.predict(X_test)

Transformar los resultados de las predicciones a valores categóticos:

In [43]:
# Invertir la transformación de las etiquetas para obtener las predicciones categóricas
predictions = label_encoder.inverse_transform(predictions.astype(int))

Se usa el dataframe id_df para concatenar y guardar los resultados en un documento csv:

In [44]:
id = pd.DataFrame({'ID': id_df})
results = pd.DataFrame({'RENDIMIENTO_GLOBAL': predictions})

In [45]:
# Guardar las predicciones en un archivo CSV
df_concat = pd.concat([id, results], axis=1)
df_concat.to_csv('submission.csv', index=False)

In [46]:
display(df_concat)

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,medio-alto
...,...,...
296781,496981,bajo
296782,209415,alto
296783,239074,alto
296784,963852,alto


Envía el documento submission.csv a la competición de kaggle:

In [47]:
!kaggle competitions submit -c udea-ai4eng-20241 -f submission.csv -m "Submission1"

100% 3.67M/3.67M [00:00<00:00, 12.7MB/s]
Successfully submitted to UDEA/ai4eng 20241 - Pruebas Saber Pro Colombia